## DENS EEG Preprocessing
This part of the project prepares brain signal data (EEG) from the DENS dataset.

1. Raw EEG signals are loaded for each participant. EEG data is very sensitive and often contains noise from things like eye blinking, muscle movement, or poor sensor contact. To reduce this noise, the signals are filtered so that only the frequency ranges that are known to come from real brain activity are kept.

2. The cleaned EEG signals are split into small time segments. This makes it easier to analyze how brain activity changes over time instead of looking at the entire recording at once.

3. For each time segment, the project measures how strong the brain signals are at different speeds, called frequency bands. These bands are linked to different mental and emotional states, such as relaxation, attention, or emotional arousal. Instead of using the raw EEG signals, these measurements are used as features because they are more stable and meaningful.

4. All of these features are then combined into one dataset and matched with the emotion labels provided by DENS.

### Key Terms

- EEG (Electroencephalography):
A way to record brain activity using sensors placed on the scalp.

- Time Segments (Windows):
Short chunks of data taken from a longer signal so changes over time can be studied.

- Frequency Bands:
Different speed ranges of brain activity that are linked to different mental states.

In [36]:
import sys
from pathlib import Path

# ensure project root is on path
p = Path.cwd()
while p != p.parent:
    if (p / "src").exists():
        PROJECT_ROOT = p
        break
    p = p.parent

sys.path.append(str(PROJECT_ROOT))
print("PROJECT_ROOT:", PROJECT_ROOT)


PROJECT_ROOT: /Users/yuliasamoilovich/Desktop/neuro_project


In [2]:
from src.eeg.dens_loader import DENS_RAW_ROOT
print("DENS_RAW_ROOT:", DENS_RAW_ROOT)
print("Contents of DENS_RAW_ROOT:", list(DENS_RAW_ROOT.iterdir()))
print("Contents of sub-mit003/eeg:", list((DENS_RAW_ROOT / "sub-mit003" / "eeg").glob("*")))


DENS_RAW_ROOT: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw
Contents of DENS_RAW_ROOT: [PosixPath('/Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit082'), PosixPath('/Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit076'), PosixPath('/Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit040'), PosixPath('/Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit014'), PosixPath('/Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit022'), PosixPath('/Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit079'), PosixPath('/Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mitb2017007'), PosixPath('/Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit023'), PosixPath('/Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit024'), PosixPath('/Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/.DS_Store'), PosixPath('/Users/yuliasamoilovich/Desktop/

In [3]:
import src.eeg.dens_loader as dl
import importlib
importlib.reload(dl)

from src.eeg.dens_loader import load_dens_subject, pick_event_label_column

raw_003, events_003 = load_dens_subject("sub-mit003")

events_003.columns
events_003.head()

label_col = pick_event_label_column(events_003)
events_003[label_col].unique()

DENS_RAW_ROOT = /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw
Looking in: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg
[sub-mit003] Loading EEG from: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg/sub-mit003_task-Emotion_eeg.set
Reading /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg/sub-mit003_task-Emotion_eeg.fdt
Reading 0 ... 674089  =      0.000 ...  2696.356 secs...


/Users/yuliasamoilovich/Desktop/neuro_project/src/eeg/dens_loader.py:25: RuntimeWarning: Unknown types found, setting as type EEG:
pns: ['ECG', 'EMG', 'EMG_2']
  raw = mne.io.read_raw_eeglab(set_path, preload=True)


[sub-mit003] Loading events from: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg/sub-mit003_task-emotion_events.tsv
[sub-mit003] raw: <RawEEGLAB | sub-mit003_task-Emotion_eeg.fdt, 132 x 674090 (2696.4 s), ~679.0 MiB, data loaded>, events rows: 130


/Users/yuliasamoilovich/Desktop/neuro_project/src/eeg/dens_loader.py:25: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True)


array(['quiz', 'Impedances Begin', 'Impedances End', 'QDone', 'baseline',
       'baseEnd', 'TrialNo', 'Fixation Loop', 'neutral_1_1', 'Valence',
       'Arousal', 'Dominance', 'Liking', 'Familiarity', 'Relevance',
       '12_2', 'click', 'Emotion_Categ', '8_3', '4_4', 'pause', nan,
       '15_5', '17_6', '16_7', 'restart', 'neutral_2_8', '7_9', '2_10',
       '24_11', 'Stop Event', 'Net Disconnected'], dtype=object)

In [4]:
stim_code = "stm"  # main event you want to epoch around
stim_events = events_003[events_003[label_col] == stim_code].copy()
len(stim_events), stim_events.head()


(0,
 Empty DataFrame
 Columns: [onset, duration, trial_type, label, description]
 Index: [])

In [5]:
import pandas as pd

stim = events_003[events_003["trial_type"] == "stm"].copy()

# split label on underscore
parts = stim["label"].str.split("_", expand=True)

# Ensure columns
parts.columns = ["stim_id", "block", "trial"]

# merge back
stim = pd.concat([stim, parts], axis=1)

stim


,onset,duration,trial_type,label,description,stim_id,block,trial
8,123630.993068,1000.0,stm,neutral_1_1,Showing the stimulation,neutral,1,1
17,148401.244450,1000.0,stm,12_2,Showing the stimulation,12,2,None
28,178767.492436,1000.0,stm,8_3,Showing the stimulation,8,3,None
37,203362.244368,1000.0,stm,4_4,Showing the stimulation,4,4,None
50,228833.722363,1000.0,stm,15_5,Showing the stimulation,15,5,None
59,260060.723072,1000.0,stm,17_6,Showing the stimulation,17,6,None
70,307649.722642,1000.0,stm,16_7,Showing the stimulation,16,7,None
87,361654.737354,1000.0,stm,neutral_2_8,Showing the stimulation,neutral,2,8
96,383754.489240,1000.0,stm,7_9,Showing the stimulation,7,9,None
108,423264.989578,1000.0,stm,2_10,Showing the stimulation,2,10,None


In [6]:
from pathlib import Path
import pandas as pd

# adjust if your path differs slightly
beh_path = Path("../data/DENS/raw/sub-mit003/beh/sub-mit003_task-Emotion_beh.tsv")
beh = pd.read_csv(beh_path, sep="\t")

beh.columns, beh.head()

(Index(['stimuliName', 'valence', 'arousal', 'dominance', 'liking',
        'familiarity', 'relevance', 'emotionCateg', 'Quadrant',
        'FivePointScale', 'MouseClick'],
       dtype='object'),
      stimuliName  valence  arousal  dominance  liking  familiarity  relevance  \
 0  neutral_1.mp4     8.97     7.95       7.04     5.0          4.0       3.98   
 1         12.m4v     9.00     9.00       8.07     5.0          5.0       4.98   
 2          8.m4v     7.01     6.00       6.50     4.0          4.0       3.25   
 3          4.mp4     1.05     7.10       5.04     1.0          2.0       1.00   
 4         15.mp4     1.00     7.95       6.02     2.0          3.0       2.50   
 
                                         emotionCateg Quadrant FivePointScale  \
 0                                                 {}      NaN            NaN   
 1  {'0': [u'Joyous: Full of happiness and joy', u...     HVHA  ['very much']   
 2                                                 {}      NaN    

In [7]:
# beh is what you already loaded
beh = beh.copy()

# make a stim_id that matches the ones from events ("neutral_1", "12", "8", ...)
beh["stim_id"] = (
    beh["stimuliName"]
    .str.replace(".mp4", "", regex=False)
    .str.replace(".m4v", "", regex=False)
)

# make sure types match
stim["stim_id"] = stim["stim_id"].astype(str)
beh["stim_id"] = beh["stim_id"].astype(str)

# keep just what we need from beh for now
beh_small = beh[["stim_id", "valence", "arousal"]]

# merge timing/events with valence & arousal
stim_meta = stim.merge(beh_small, on="stim_id", how="left")

stim_meta[["label", "stim_id", "valence", "arousal"]].head()


,label,stim_id,valence,arousal
0,neutral_1_1,neutral,NaN,NaN
1,12_2,12,9.00,9.00
2,8_3,8,7.01,6.00
3,4_4,4,1.05,7.10
4,15_5,15,1.00,7.95


In [8]:
stim_meta[["valence", "arousal"]].isna().sum()

valence    2
arousal    2
dtype: int64

In [9]:
import pandas as pd
def classify_state(row):
    stim = row["stim_id"]
    v = row["valence"]
    a = row["arousal"]

    # 0. Explicitly neutral → baseline
    if stim == "neutral":
        return "baseline"

    # 1. Numeric rules when we have ratings
    if pd.notna(a) and pd.notna(v):
        # high arousal & low valence → stress
        if a >= 6.0 and v <= 3.0:
            return "stress"

        # high arousal & high valence → amusement
        if a >= 6.0 and v >= 6.0:
            return "amusement"

        # everything else → baseline-ish
        return "baseline"

    # 2. Fallback: if no ratings at all → baseline
    return "baseline"


In [10]:
stim_meta["state"] = stim_meta.apply(classify_state, axis=1)
stim_meta["state"].value_counts(dropna=False)

state
stress       5
baseline     4
amusement    2
Name: count, dtype: int64

In [11]:
stim_meta[stim_meta["state"].isna()][["stim_id", "valence", "arousal", "label"]]

,stim_id,valence,arousal,label


In [12]:
label_state_map = (
    stim_meta[["label", "state"]]
    .dropna()
    .drop_duplicates()
    .set_index("label")["state"]
    .to_dict()
)

label_state_map

{'neutral_1_1': 'baseline',
 '12_2': 'amusement',
 '8_3': 'amusement',
 '4_4': 'stress',
 '15_5': 'stress',
 '17_6': 'baseline',
 '16_7': 'stress',
 'neutral_2_8': 'baseline',
 '7_9': 'baseline',
 '2_10': 'stress',
 '24_11': 'stress'}

In [13]:
from src.eeg.dens_preprocessing import build_dens_msff_for_subject

df_003 = build_dens_msff_for_subject(
    "sub-mit003",
    win_sec=4.0,
    step_sec=2.0,
    min_overlap_ratio=0.2,   # more lenient
)

len(df_003), df_003.columns


DENS_RAW_ROOT = /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw
Looking in: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg
[sub-mit003] Loading EEG from: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg/sub-mit003_task-Emotion_eeg.set
Reading /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg/sub-mit003_task-Emotion_eeg.fdt
Reading 0 ... 674089  =      0.000 ...  2696.356 secs...


/Users/yuliasamoilovich/Desktop/neuro_project/src/eeg/dens_loader.py:25: RuntimeWarning: Unknown types found, setting as type EEG:
pns: ['ECG', 'EMG', 'EMG_2']
  raw = mne.io.read_raw_eeglab(set_path, preload=True)
/Users/yuliasamoilovich/Desktop/neuro_project/src/eeg/dens_loader.py:25: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True)


[sub-mit003] Loading events from: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg/sub-mit003_task-emotion_events.tsv
[sub-mit003] raw: <RawEEGLAB | sub-mit003_task-Emotion_eeg.fdt, 132 x 674090 (2696.4 s), ~679.0 MiB, data loaded>, events rows: 130
[sub-mit003] Converting event onsets/durations from samples to seconds
Made 1347 windows of 4.0s with step 2.0s
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)


(36,
 Index(['subject_id', 'layer', 'start', 'end', 'raw_label', 'label',
        'delta_power_mean', 'delta_power_std', 'theta_power_mean',
        'theta_power_std', 'alpha_power_mean', 'alpha_power_std',
        'beta_power_mean', 'beta_power_std', 'gamma_power_mean',
        'gamma_power_std', 'alpha_asym_F4_minus_F3', 'label_code'],
       dtype='object'))

In [16]:
len(df_003)
df_003["label"].value_counts(dropna=False)


label
stress       16
baseline     13
amusement     7
Name: count, dtype: int64

In [17]:
df_003["label"].value_counts()
df_003[["start", "end", "raw_label", "label"]].head()

,start,end,raw_label,label
0,492.0,496.0,neutral_1_1,baseline
1,494.0,498.0,neutral_1_1,baseline
2,496.0,500.0,neutral_1_1,baseline
3,592.0,596.0,12_2,amusement
4,594.0,598.0,12_2,amusement


In [18]:
# Discover all DENS subjects
from pathlib import Path
dens_root = Path("../data/DENS/raw")
subject_dirs = sorted([d.name for d in dens_root.iterdir() if d.is_dir() and d.name.startswith("sub-")])
print("Available DENS subjects:", subject_dirs)

Available DENS subjects: ['sub-mit003', 'sub-mit004', 'sub-mit007', 'sub-mit014', 'sub-mit017', 'sub-mit022', 'sub-mit023', 'sub-mit024', 'sub-mit032', 'sub-mit035', 'sub-mit039', 'sub-mit040', 'sub-mit051', 'sub-mit052', 'sub-mit054', 'sub-mit061', 'sub-mit067', 'sub-mit072', 'sub-mit074', 'sub-mit076', 'sub-mit079', 'sub-mit080', 'sub-mit081', 'sub-mit082', 'sub-mit096', 'sub-mit097', 'sub-mit099', 'sub-mit104', 'sub-mit106', 'sub-mit107', 'sub-mit108', 'sub-mit111', 'sub-mit113', 'sub-mit114', 'sub-mit116', 'sub-mit117', 'sub-mit121', 'sub-mit122', 'sub-mit123', 'sub-mitb2017007']


In [29]:
# Build MSFF for all subjects (with error handling)
from src.eeg.dens_preprocessing import build_all_dens_msff

# First, check which subjects have valid EEG files
from pathlib import Path
dens_root = Path("../data/DENS/raw")
valid_subjects = []
for sid in subject_dirs:
    eeg_dir = dens_root / sid / "eeg"
    set_files = list(eeg_dir.glob("*_task-Emotion_eeg.set"))
    if set_files:
        valid_subjects.append(sid)
        print(f"✓ {sid}: found {set_files[0].name}")
    else:
        print(f"✗ {sid}: no EEG file found")

print(f"\nValid subjects: {valid_subjects}")

✓ sub-mit003: found sub-mit003_task-Emotion_eeg.set
✓ sub-mit004: found sub-mit004_task-Emotion_eeg.set
✓ sub-mit007: found sub-mit007_task-Emotion_eeg.set
✓ sub-mit014: found sub-mit014_task-Emotion_eeg.set
✓ sub-mit017: found sub-mit017_task-Emotion_eeg.set
✓ sub-mit022: found sub-mit022_task-Emotion_eeg.set
✓ sub-mit023: found sub-mit023_task-Emotion_eeg.set
✓ sub-mit024: found sub-mit024_task-Emotion_eeg.set
✓ sub-mit032: found sub-mit032_task-Emotion_eeg.set
✓ sub-mit035: found sub-mit035_task-Emotion_eeg.set
✓ sub-mit039: found sub-mit039_task-Emotion_eeg.set
✓ sub-mit040: found sub-mit040_task-Emotion_eeg.set
✓ sub-mit051: found sub-mit051_task-Emotion_eeg.set
✓ sub-mit052: found sub-mit052_task-Emotion_eeg.set
✓ sub-mit054: found sub-mit054_task-Emotion_eeg.set
✓ sub-mit061: found sub-mit061_task-Emotion_eeg.set
✓ sub-mit067: found sub-mit067_task-Emotion_eeg.set
✓ sub-mit072: found sub-mit072_task-Emotion_eeg.set
✓ sub-mit074: found sub-mit074_task-Emotion_eeg.set
✓ sub-mit076

In [34]:
import importlib
import src.eeg.dens_preprocessing as dp
importlib.reload(dp)
from src.eeg.dens_preprocessing import build_all_dens_msff, validate_dens_msff

df_all = build_all_dens_msff(
    subject_ids=valid_subjects,
    save=True,
    filename="dens_brain.csv"
)

print(f"Dataset shape: {df_all.shape}")
print("\nLabel distribution:")
print(df_all["label"].value_counts(dropna=False))


=== Processing sub-mit003 ===
DENS_RAW_ROOT = /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw
Looking in: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg
[sub-mit003] Loading EEG from: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg/sub-mit003_task-Emotion_eeg.set
Reading /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg/sub-mit003_task-Emotion_eeg.fdt
Reading 0 ... 674089  =      0.000 ...  2696.356 secs...


/Users/yuliasamoilovich/Desktop/neuro_project/src/eeg/dens_loader.py:25: RuntimeWarning: Unknown types found, setting as type EEG:
pns: ['ECG', 'EMG', 'EMG_2']
  raw = mne.io.read_raw_eeglab(set_path, preload=True)


[sub-mit003] Loading events from: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit003/eeg/sub-mit003_task-emotion_events.tsv
[sub-mit003] raw: <RawEEGLAB | sub-mit003_task-Emotion_eeg.fdt, 132 x 674090 (2696.4 s), ~679.0 MiB, data loaded>, events rows: 130
[sub-mit003] Converting event onsets/durations from samples to seconds
Made 1347 windows of 4.0s with step 2.0s
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)


/Users/yuliasamoilovich/Desktop/neuro_project/src/eeg/dens_loader.py:25: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True)


Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
[sub-mit003] MSFF rows: 19 (kept=19, skipped=1328, labels={'baseline': 9, 'stress': 7, 'amusement': 3})
✓ sub-mit003: 19 rows added

=== Processing sub-mit004 ===
DENS_RAW_ROOT = /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw
Looking in: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit004/eeg
[sub-mit004] Loading EEG from: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit004/eeg/sub-mit004_task-Emotion_eeg.set
Reading /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mit004/eeg/sub-mit004_task-Emotion_eeg.fdt
Reading 0 ... 430526  =      0.000 ...  1722.104 secs...


...

/Users/yuliasamoilovich/Desktop/neuro_project/src/eeg/dens_loader.py:25: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(set_path, preload=True)
/Users/yuliasamoilovich/Desktop/neuro_project/src/eeg/dens_loader.py:25: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True)


Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
[sub-mit123] MSFF rows: 18 (kept=18, skipped=643, labels={'baseline': 15, 'stress': 3})
✓ sub-mit123: 18 rows added

=== Processing sub-mitb2017007 ===
DENS_RAW_ROOT = /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw
Looking in: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mitb2017007/eeg
[sub-mitb2017007] Loading EEG from: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mitb2017007/eeg/sub-mitb2017007_task-Emotion_eeg.set
Reading /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mitb2017007/eeg/sub-mitb2017007_task-Emotion_eeg.fdt
Reading 0 ... 391082  =      0.000 ...  1564.328 secs...


/Users/yuliasamoilovich/Desktop/neuro_project/src/eeg/dens_loader.py:25: RuntimeWarning: Unknown types found, setting as type EEG:
pns: ['ECG', 'EMG', 'EMG_2']
  raw = mne.io.read_raw_eeglab(set_path, preload=True)


[sub-mitb2017007] Loading events from: /Users/yuliasamoilovich/Desktop/neuro_project/data/DENS/raw/sub-mitb2017007/eeg/sub-mitb2017007_task-emotion_events.tsv
[sub-mitb2017007] raw: <RawEEGLAB | sub-mitb2017007_task-Emotion_eeg.fdt, 132 x 391083 (1564.3 s), ~394.0 MiB, data loaded>, events rows: 127
[sub-mitb2017007] Converting event onsets/durations from samples to seconds
Made 781 windows of 4.0s with step 2.0s
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
[sub-

/Users/yuliasamoilovich/Desktop/neuro_project/src/eeg/dens_loader.py:25: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True)


In [35]:
import importlib
import src.eeg.dens_preprocessing as dp
importlib.reload(dp)
from src.eeg.dens_preprocessing import validate_dens_msff

# Validate the full dataset
summary = validate_dens_msff(df_all)

for key, val in summary.items():
    print(f"\n{key}:")
    if isinstance(val, dict):
        for k2, v2 in val.items():
            print(f"  {k2}: {v2}")
    else:
        print(f"  {val}")

# Check class balance
print("\nLabel distribution:")
print(df_all["label"].value_counts(dropna=False))

print("\nFeature sample:")
print(df_all.head())


n_rows:
  141

n_subjects:
  8

label_dist:
  baseline: 88
  stress: 30
  amusement: 23

n_features:
  11

n_nans_per_col:
  subject_id: 0
  layer: 0
  start: 0
  end: 0
  raw_label: 0
  label: 0
  delta_power_mean: 0
  delta_power_std: 0
  theta_power_mean: 0
  theta_power_std: 0
  alpha_power_mean: 0
  alpha_power_std: 0
  beta_power_mean: 0
  beta_power_std: 0
  gamma_power_mean: 0
  gamma_power_std: 0
  alpha_asym_F4_minus_F3: 141
  label_code: 0

Label distribution:
label
baseline     88
stress       30
amusement    23
Name: count, dtype: int64

Feature sample:
   subject_id  layer  start    end    raw_label      label  delta_power_mean  \
0  sub-mit003  brain  494.0  498.0  neutral_1_1   baseline      1.438120e-10   
1  sub-mit003  brain  496.0  500.0  neutral_1_1   baseline      6.111161e-11   
2  sub-mit003  brain  594.0  598.0         12_2  amusement      1.959582e-10   
3  sub-mit003  brain  714.0  718.0          8_3  amusement      8.661912e-11   
4  sub-mit003  brain  716.

In [ ]:
# Build complete emotion-to-state mapping across ALL subjects
from pathlib import Path
import pandas as pd

dens_root = Path("../data/DENS/raw")
all_mappings = {}

for sid in valid_subjects:  # ALL subjects, not just [:5]
    try:
        beh_path = dens_root / sid / "beh" / f"{sid}_task-Emotion_beh.tsv"
        events_path = dens_root / sid / "eeg" / f"{sid}_task-emotion_events.tsv"
        
        if not beh_path.exists() or not events_path.exists():
            continue
        
        beh = pd.read_csv(beh_path, sep="\t")
        events = pd.read_csv(events_path, sep="\t")
        
        # Map stim_id from events to valence/arousal from beh
        stim = events[events["trial_type"] == "stm"].copy()
        parts = stim["label"].str.split("_", expand=True)
        parts.columns = ["stim_id", "block", "trial"]
        stim = pd.concat([stim, parts], axis=1)
        
        beh["stim_id"] = beh["stimuliName"].str.replace(".mp4", "", regex=False).str.replace(".m4v", "", regex=False)
        stim["stim_id"] = stim["stim_id"].astype(str)
        beh["stim_id"] = beh["stim_id"].astype(str)
        
        stim_meta = stim.merge(beh[["stim_id", "valence", "arousal"]], on="stim_id", how="left")
        
        for _, row in stim_meta.iterrows():
            label = row["label"]
            v = row["valence"]
            a = row["arousal"]
            
            if pd.isna(v) or pd.isna(a):
                state = "baseline" if "neutral" in str(label) else "baseline"
            elif a >= 6.0 and v <= 3.0:
                state = "stress"
            elif a >= 6.0 and v >= 6.0:
                state = "amusement"
            else:
                state = "baseline"
            
            all_mappings[label] = state
    except Exception as e:
        print(f"Error processing {sid}: {e}")

print(f"Total unique labels: {len(all_mappings)}\n")
print("EMOTION_TO_STATE = {")
for k, v in sorted(all_mappings.items()):
    print(f"    '{k}': '{v}',")
print("}")

Total unique labels: 145

EMOTION_TO_STATE = {
    '12_10': 'amusement',
    '12_2': 'amusement',
    '12_3': 'amusement',
    '12_34': 'amusement',
    '12_4': 'amusement',
    '12_7': 'amusement',
    '12_8': 'amusement',
    '12_9': 'baseline',
    '13_10': 'amusement',
    '13_11': 'baseline',
    '13_2': 'baseline',
    '13_3': 'amusement',
    '13_4': 'baseline',
    '13_5': 'baseline',
    '13_6': 'amusement',
    '15_10': 'stress',
    '15_2': 'amusement',
    '15_3': 'stress',
    '15_32': 'baseline',
    '15_4': 'baseline',
    '15_5': 'baseline',
    '15_6': 'baseline',
    '15_7': 'stress',
    '15_8': 'amusement',
    '15_9': 'baseline',
    '16_10': 'baseline',
    '16_11': 'stress',
    '16_2': 'stress',
    '16_3': 'baseline',
    '16_36': 'stress',
    '16_5': 'stress',
    '16_6': 'stress',
    '16_7': 'stress',
    '16_9': 'stress',
    '17_10': 'baseline',
    '17_11': 'baseline',
    '17_2': 'stress',
    '17_5': 'stress',
    '17_6': 'baseline',
    '17_7': 'basel